In [2]:
import pandas as pd
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
inspections_file = "csvFiles/HealthInspections.csv"
inspections_df = pd.read_csv(inspections_file)
inspections_df.head()

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
0,DAJ00E07B,2017-12-29T00:00:00,HABITAT COFFEE SHOP,95,A,1,ROUTINE INSPECTION,EE0000923,3708 N EAGLE ROCK BLVD,LOS ANGELES,FA0170465,CA,90065,OW0178123,GLASSELL COFFEE SHOP LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,HABITAT COFFEE SHOP,ACTIVE,PR0160774
1,DAQOKRFZB,2017-12-29T00:00:00,REILLY'S,92,A,1,ROUTINE INSPECTION,EE0000633,100 WORLD WAY # 120,LOS ANGELES,FA0244690,CA,90045,OW0208441,"AREAS SKYVIEW LAX JV, LLC",RESTAURANT (0-30) SEATS MODERATE RISK,1631,REILLY'S,ACTIVE,PR0193026
2,DASJI4LUR,2017-12-29T00:00:00,STREET CHURROS,93,A,1,ROUTINE INSPECTION,EE0000835,6801 HOLLYWOOD BLVD # 253,LOS ANGELES,FA0224109,CA,90028,OW0228670,"STREETCHURROS, INC",RESTAURANT (0-30) SEATS LOW RISK,1630,STREET CHURROS,ACTIVE,PR0179282
3,DAWVA0CY3,2017-12-29T00:00:00,RIO GENTLEMANS CLUB,93,A,1,ROUTINE INSPECTION,EE0000958,13124 S FIGUEROA ST,LOS ANGELES,FA0046462,CA,90064,OW0011830,FIGUEROA GROUP INC,RESTAURANT (61-150) SEATS LOW RISK,1636,RIO GENTLEMANS CLUB,ACTIVE,PR0044776
4,DAKFCHD0L,2017-12-29T00:00:00,LE PAIN QUOTIDIEN,93,A,1,ROUTINE INSPECTION,EE0000629,13050 SAN VICENTE BLVD STE 114,LOS ANGELES,FA0034788,CA,90049,OW0028928,PQ SAN VICENTE INC.,RESTAURANT (61-150) SEATS HIGH RISK,1638,LE PAIN QUOTIDIEN,ACTIVE,PR0017456


In [ ]:
json_file = "jsonFiles/yelp_academic_dataset_business.json"
violations_df = pd.read_json(json_file, lines=True)
violations_df.groupby("city")